# Sensor Position Optimization

### BMW Quantum Computing Challenge

#### Valter Uotila, Unified Database Management Systems
#### Sardana Ivanova, Discovery Research Group

#### Department of Computer Science, University of Helsinki

## Remarks about the implementation

We have been learning, testing and developing some basic quantum algorithms using D-waves own API but we are not very experienced with details related to these quantum computers.

Unfortunately, it seems that Amazon Bracket does not yet support D-waves hybrid solvers. We might be wrong but we could not figure out how to access, for example, LeapHybridSampler using Bracket. It seems that BracketDWaveSampler and BracketSampler are using the standard DWaveSampler (https://docs.ocean.dwavesys.com/projects/system/en/stable/reference/samplers.html). Because our solution is based on LeapHybridSampler, it is not possible to run it in Bracket. Maybe in the future Amazon will implement something like BracketDWaveHybridSampler which offers the functionality of LeapHybridSampler. Anyway, we are positively impressed how Bracket collects various quantum computing resources together.

## Initializing parameters

### Importing D-wave packages

Note that in order to run the codes here, you need to be able to successfully access D-wave quantum cloud computing resources. You can see more info at https://cloud.dwavesys.com/. We are used to develop with D-waves developer's plan which includes 1 min of quantum computing time per month.

In [209]:
import dimod

from dwave.system import LeapHybridSampler

import json
import csv

import itertools
import os
import math

import numpy as np
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from sympy import *

from ipynb.fs.defs.overlap_3D import overlap

notebook_path = os.path.abspath("main_3D_connecting_Dwave_Leap.ipynb")

### Global parameters

In [210]:
car_sample_accuracy = 100

angle_accuracy = 30

variables = dict()

### Importing sensors

In [211]:
abs_sensors_file_path = os.path.join(os.path.dirname(notebook_path), "3d_example_data/3D_sensors_set_1.json")

sensor_types = {0: 'lidar', 1: 'radar', 2: 'camera', 3: 'ultrasound'}

f = open(abs_sensors_file_path)

sensor_root = json.load(f)
sensors = sensor_root["sensors"]

print(sensors)

[{'id': 1, 'type': 1, 'view': {'angle': {'horizontal': 10, 'vertical': 50}, 'range': 200}, 'price': 100}, {'id': 2, 'type': 2, 'view': {'angle': {'horizontal': 10, 'vertical': 50}, 'range': 500}, 'price': 200}, {'id': 3, 'type': 3, 'view': {'angle': {'horizontal': 10, 'vertical': 50}, 'range': 700}, 'price': 100}, {'id': 4, 'type': 1, 'view': {'angle': {'horizontal': 10, 'vertical': 50}, 'range': 1000}, 'price': 10}, {'id': 5, 'type': 2, 'view': {'angle': {'horizontal': 10, 'vertical': 50}, 'range': 1000}, 'price': 100}]


### Initializing allowed sensor positions on car

Here we utilize the data BMW provided. Compared to the first-round 2D version, the car model in this second-round version is based on the allowed sensor positions data. In order to create variables, we need to sample some points from the surfaces. To sample points, we need some easier format than vector representation. Thus we calculate the plane equation for the surfaces. The equation allows us to sample points when we are creating the variables.

In [212]:
def plane_equation(c1, c2, c3):
    p1 = np.array([c1[0], c1[1], c1[2]])
    p2 = np.array([c2[0], c2[1], c2[2]])
    p3 = np.array([c3[0], c3[1], c3[2]])
    
    # These two vectors are in the plane
    v1 = p3 - p1
    v2 = p2 - p1
    
    # cross product is a vector normal to the plane
    cp = np.cross(v1, v2)
    a, b, c = cp
    
    # This evaluates a * x3 + b * y3 + c * z3 which equals d
    d = np.dot(cp, p3)
    
    #print('Equation is {0}x + {1}y + {2}z = {3}'.format(a, b, c, d))
    
    #plot_region(a, b, c, d)
    
    return { 'span1': v1, 'span2': v2, 'normal_vector': cp, 'x': a, 'y': b, 'z': c, 'd': d }

def plot_region(a, b, c, d):
    x = np.linspace(-1,1,10)
    y = np.linspace(-1,1,10)

    X,Y = np.meshgrid(x,y)
    if c != 0:
        Z = (d - a*X - b*Y) / c

        fig = plt.figure()
        ax = fig.gca(projection='3d')

        surf = ax.plot_surface(X, Y, Z)
        
def scatter_plot(points):
    fig = plt.figure()
    #ax = pyplot.axes(projection='3d')
    #ax = fig.add_subplot(111, projection='3d')
    ax = Axes3D(fig)
    
    x_vals = [c[0] for c in points]
    y_vals = [c[1] for c in points]
    z_vals = [c[2] for c in points]
    
    #ax.scatter3D(x_vals, y_vals, z_vals, c=z_vals);
    
    ax.scatter(x_vals, y_vals, z_vals)
    #ax.set_xlabel('X Label')
    #ax.set_ylabel('Y Label')
    #ax.set_zlabel('Z Label')
    pyplot.show()
    
    

In [213]:
abs_position_file_path = os.path.join(os.path.dirname(notebook_path), "sensor_position_data/allowed_sensor_positions.csv")

allowed_sensor_positions = dict()

with open(abs_position_file_path, newline='') as csvfile:
    records = iter(csv.DictReader(csvfile, delimiter=';'))
    next(records)
    for row in records:
        c1 = (int(row["x1"]), int(row["y1"]), int(row["z1"]))
        c2 = (int(row["x2"]), int(row["y2"]), int(row["z2"]))
        c3 = (int(row["x3"]), int(row["y3"]), int(row["z3"]))
        c4 = (int(row["x4"]), int(row["y4"]), int(row["z4"]))
        
        region = row["Region"].lower()
        
        allowed_sensor_positions[region] = dict()
        allowed_sensor_positions[region]["corners"] = [c1, c2, c3, c4]
        allowed_sensors = [x.lower() for x in row["Allowed Sensors"].split(", ")]
        allowed_sensor_positions[region]["allowed_sensors"] = allowed_sensors
        
        equation = plane_equation(c1, c2, c3)
        allowed_sensor_positions[region]["equation"] = equation
        
        intervals = dict()
            
        if equation["x"] == 0:
            if c1[0] < c2[0]:
                intervals["x"] = range(c1[0], c2[0], car_sample_accuracy)
            elif c1[0] > c2[0]:
                intervals["x"] = range(c2[0], c1[0], car_sample_accuracy)
            elif c1[0] < c3[0]:
                intervals["x"] = range(c1[0], c3[0], car_sample_accuracy)
            elif c1[0] > c3[0]:
                intervals["x"] = range(c3[0], c1[0], car_sample_accuracy)
                
        if equation["y"] == 0:
            if c1[1] < c2[1]:
                intervals["y"] = range(c1[1], c2[1], car_sample_accuracy)
            elif c1[1] > c2[1]:
                intervals["y"] = range(c2[1], c1[1], car_sample_accuracy)
            elif c1[1] < c3[1]:
                intervals["y"] = range(c1[1], c3[1], car_sample_accuracy)
            elif c1[1] > c3[1]:
                intervals["y"] = range(c3[1], c1[1], car_sample_accuracy)
                
        if equation["z"] == 0:
            if c1[2] < c2[2]:
                intervals["z"] = range(c1[2], c2[2], car_sample_accuracy)
            elif c1[2] > c2[2]:
                intervals["z"] = range(c2[2], c1[2], car_sample_accuracy)
            elif c1[2] < c3[2]:
                intervals["z"] = range(c1[2], c3[2], car_sample_accuracy)
            elif c1[2] > c3[2]:
                intervals["z"] = range(c3[2], c1[2], car_sample_accuracy)
                
        allowed_sensor_positions[region]["fixed_intervals"] = intervals

for elem in allowed_sensor_positions:
    print(elem, allowed_sensor_positions[elem])
        

bonnet {'corners': [(-1400, 1000, 1200), (-1400, -1000, 1200), (0, -1000, 1000), (0, 1000, 1000)], 'allowed_sensors': ['lidar', 'camera'], 'equation': {'span1': array([ 1400, -2000,  -200]), 'span2': array([    0, -2000,     0]), 'normal_vector': array([ -400000,        0, -2800000]), 'x': -400000, 'y': 0, 'z': -2800000, 'd': 1494967296}, 'fixed_intervals': {'y': range(-1000, 1000, 100)}}
side mirror left {'corners': [(-1400, 1150, 1320), (-1400, 950, 1320), (-1400, 950, 1200), (-1400, 1150, 1200)], 'allowed_sensors': ['lidar', 'camera'], 'equation': {'span1': array([   0, -200, -120]), 'span2': array([   0, -200,    0]), 'normal_vector': array([-24000,      0,      0]), 'x': -24000, 'y': 0, 'z': 0, 'd': 33600000}, 'fixed_intervals': {'y': range(950, 1150, 100), 'z': range(1200, 1320, 100)}}
side mirror right {'corners': [(-1400, -950, 1320), (-1400, -1150, 1320), (-1400, -1150, 1200), (-1400, -950, 1200)], 'allowed_sensors': ['lidar', 'camera'], 'equation': {'span1': array([   0, -200

### Initializing variables

Compared to the first-round, variables are updated to be triples (x, y, i) where x and y are points in the 3D-space and i refers to the sensor id. We fix the point x so that it belongs to some of the allowed sensor positions on the car's surface. The point y is selected from the environment so that the distance between x and y is range R_i for sensor i.

We are dealing with different cases depending on how the car surface is positioned in the space.

In [214]:
def sample_from_car_surface(corners, equation, fixed_intervals, car_sample_accuracy):
    sample = list()
    
    # This is the simple case that the plane is parallel with some of the three axis. 
    # Thus the parallel axis stays constant.
    # For example, side mirror, back, trunk and sides are parallel to one of the axis
    
    if len(fixed_intervals) == 2:
        if 'x' in fixed_intervals and 'y' in fixed_intervals:
            
            z = corners[0][2]
            
            for x in fixed_intervals['x']:
                for y in fixed_intervals['y']:
                    sample.append((x, y, z))
                    
        elif 'x' in fixed_intervals and 'z' in fixed_intervals:
            
            y = corners[0][1]
            
            for x in fixed_intervals['x']:
                for z in fixed_intervals['z']:
                    sample.append((x, y, z))
                    
        elif 'y' in fixed_intervals and 'z' in fixed_intervals:
            
            x = corners[0][0]
            
            for y in fixed_intervals['y']:
                for z in fixed_intervals['z']:
                    sample.append((x, y, z))

    elif len(fixed_intervals) == 1:
        
        if 'x' in fixed_intervals:
            y, z = symbols('y z')
            expr = equation['y']*y + equation['z']*z - equation['d']
            y_interval = None
            
            c1 = corners[0][1]
            c2 = corners[1][1]
            c3 = corners[2][1]
            
            if c1 < c2:
                y_interval = range(c1, c2, car_sample_accuracy)
            elif c1 > c2:
                y_interval = range(c2, c1, car_sample_accuracy)
            elif c1 < c3:
                y_interval = range(c1, c3, car_sample_accuracy)
            elif c1 > c3:
                y_interval = range(c3, c1, car_sample_accuracy)
        
            
            for x in fixed_intervals['x']:
                for y_var in y_interval:
                    y_expr = expr.subs(y, y_var)
                    z = math.floor(solve(y_expr)[0])
                    sample.append((x, y_var, z))
                    #print((x, y_var, z))
                x += car_sample_accuracy
                    
        elif 'y' in fixed_intervals:
            x, z = symbols('x z')
            expr = equation['x']*x + equation['z']*z - equation['d']
            x_interval = None
            
            c1 = corners[0][0]
            c2 = corners[1][0]
            c3 = corners[2][0]
            
            if c1 < c2:
                x_interval = range(c1, c2, car_sample_accuracy)
            elif c1 > c2:
                x_interval = range(c2, c1, car_sample_accuracy)
            elif c1 < c3:
                x_interval = range(c1, c3, car_sample_accuracy)
            elif c1 > c3:
                x_interval = range(c3, c1, car_sample_accuracy)
        
            
            for y in fixed_intervals['y']:
                for x_var in x_interval:
                    x_expr = expr.subs(x, x_var)
                    z = math.floor(solve(x_expr)[0])
                    sample.append((x_var, y, z))
                    #print((x_var, y, z))
                y += car_sample_accuracy
            
        elif 'z' in fixed_intervals:
                
            x, y = symbols('x y')
            expr = equation['x']*x + equation['y']*y - equation['d']
            x_interval = None

            c1 = corners[0][0]
            c2 = corners[1][0]
            c3 = corners[2][0]

            if c1 < c2:
                x_interval = range(c1, c2, car_sample_accuracy)
            elif c1 > c2:
                x_interval = range(c2, c1, car_sample_accuracy)
            elif c1 < c3:
                x_interval = range(c1, c3, car_sample_accuracy)
            elif c1 > c3:
                x_interval = range(c3, c1, car_sample_accuracy)


            for z in fixed_intervals['z']:
                for x_var in x_interval:
                    x_expr = expr.subs(x, x_var)
                    y = math.floor(solve(x_expr)[0])
                    #print(y)
                    sample.append((x_var, y, z))
                    #print((x_var, y, z))
                z += car_sample_accuracy
        
    return sample



In [ ]:
def euclidean_3d_distance(x, y):
    return math.floor(math.sqrt(pow((x[0] - y[0]), 2) + pow((x[1] - y[1]), 2) + pow((x[2] - y[2]), 2)))

# Sample points from the cars surface
# In the optimal situation we could sample points with accuracy factor 1. This would produce a huge number of points but the system cannot scale that well.

for sensor in sensors:
    variables[sensor["id"]] = dict()
    for pos_name in allowed_sensor_positions:
        pos = allowed_sensor_positions[pos_name]
        #print(sensor_types)
        if sensor_types[sensor["type"]] in pos["allowed_sensors"]:
            variables[sensor["id"]][pos_name] = list()
            srange = sensor["view"]["range"]
            corners = pos["corners"]
            equation = pos["equation"]
            fixed_intervals = pos["fixed_intervals"]
            
            car_sample = sample_from_car_surface(corners, equation, fixed_intervals, car_sample_accuracy)
            
            #print(pos_name)
            #print(car_sample)
            #print()
            
            #scatter_plot(car_sample)
        
            for car_point in car_sample:
                env_points = list()
                for h_angle in range(0, 180, angle_accuracy):
                    for v_angle in range(0, 90, angle_accuracy):
                        
                        rad_h_angle = math.radians(h_angle)
                        rad_v_angle = math.radians(v_angle)
                        
                        #print(car_point)
                        
                        #print(math.floor(car_point[0] + srange*math.sin(rad_v_angle)))
                        
                        normal_vector = equation['normal_vector']
                        spanning_vector_on_plane1 = equation['span1']
                        spanning_vector_on_plane2 = equation['span2']
                        
                        point_in_environment = np.array([0,0,0])
                        car_point_vector = np.array(car_point)
                        
                        # Move along plane
                        point_in_environment += math.floor(srange*math.sin(rad_v_angle))*spanning_vector_on_plane1
                        
                        # Move along plane
                        point_in_environment += math.floor(srange*math.sin(math.pi/2 - rad_v_angle))*spanning_vector_on_plane2
                        
                        # Move to orthonogal direction of plane
                        point_in_environment += math.floor(srange*math.sin(rad_h_angle/2))*normal_vector
                        
                        #print(point_in_environment)
                        
                        #env_points.append(point_in_environment)
                        #scatter_plot(env_points)
            
                        b_variable = (car_point, point_in_environment, sensor["id"])
                        variables[sensor["id"]][pos_name].append(b_variable)
            

#print(len(variables))


## Constructing quadratic unconstrained binary optimization model

In [5]:
vartype = dimod.BINARY
main_bqm = dimod.BinaryQuadraticModel({}, {}, 0.0, vartype)

# Initializing variables
# Variables are triples (x, y, i) where x and y are coordinate points in the space
# and i is the index of the sensor.

def create_variables(start_x, end_x, start_y, end_y, sensors):
    variables = []
    for x in range(start_x, end_x + 1):
        for y in range(start_y, end_y + 1):
            if (x,y) not in car_grid:
                variables.append((x, y))
    return variables

env = create_variables(environment["v1"][0], environment["v2"][0], environment["v3"][1], environment["v2"][1], sensors)

# print(len(env))

# Next we collect the variables which are assigned to the car surface

car_surface = []

for pos in car["positions"]:
    if pos["x1"] == pos["x2"]:
        x = pos["x1"]
        for y in range(pos["y1"], pos["y2"] + 1):
            car_surface.append((x, y))
    elif pos["y1"] == pos["y2"]:
        y = pos["y1"]
        for x in range(pos["x1"], pos["x2"] + 1):
            car_surface.append((x, y))
    else:
        raise ValueError('The allowed position on the car surface is not a line.')
        
# print(car_surface)

The following functions enable us to append or initialize coefficients for the variables in the BQM. The distance function implements the ordinary Euclidean distance.

In [6]:
def append_linear_safe(variable, value, linear_dict):
    if variable in linear_dict.keys():
        linear_dict[variable] = linear_dict[variable] + value
    else:
        linear_dict[variable] = value

def append_quadratic_safe(variable, value, quadratic_dict):
    if variable in quadratic_dict.keys():
        quadratic_dict[variable] = quadratic_dict[variable] + value
    else:
        quadratic_dict[variable] = value

### Constraint 1: selecting sufficiently sensors to cover the environment

Every binary quadratic function which is part of the model contains four parameters: linear terms, quadratic terms, offset (constant) and variable type. Variable type is always BINARY since we are using QUBO. If we would use Ising, we set variable type to be SPIN.

In [7]:
# Encoding constraint H1

A1 = 1
linear_h1 = {}
quadratic_h1 = {}
offset_h1 = 0.0

for sensor in sensors:
    for y in env:
        
        var_y = (y[0], y[1], y[2], sensor["id"])
        append_linear_safe(var_y, 1, linear_h1)
        
        for x in car_surface:
            
            var_x = (x[0], x[1], x[2], sensor["id"])
            
            append_linear_safe(var_x, 1, linear_h1)
            append_quadratic_safe((var_x, var_y), -2, quadratic_h1)
            
        for tupl in itertools.combinations(car_surface, 2):
            
            var_1 = (tupl[0][0], tupl[0][1], sensor["id"])
            var_2 = (tupl[1][0], tupl[1][1], sensor["id"])
            
            append_quadratic_safe((var_1, var_2), 2, quadratic_h1)
            
bqm_h1 = dimod.BinaryQuadraticModel(linear_h1, quadratic_h1, offset_h1, vartype)
bqm_h1.scale(A1)
main_bqm.update(bqm_h1)
            
#for elem in main_bqm.linear:
#        print(elem, main_bqm.linear[elem])

#for elem in main_bqm.quadratic:
#    print(elem, main_bqm.quadratic[elem])
            

### Constraint 2: optimizing overlap of sensor views

In [8]:
# Encoding constraint H3

A3 = 1
linear_h3 = {}
quadratic_h3 = {}
offset_h3 = 1

for sensor in sensors:
    for x in car_surface:
        for y in env:
            constant = float(pow(distance(x, y) - sensor["view"]["range"], 2)) + 1
            
            # print(constant)
            #if constant != 1:
            #    constant = pow(distance(x, y) + 1 - sensor["view"]["range"] + 1, 2)
            
            var_x = (x[0], x[1], x[2], sensor["id"])
            var_y = (y[0], y[1], y[2], sensor["id"])
            
            #append_linear_safe(var_x, pow(constant, 2), linear_h3)
            #append_linear_safe(var_y, pow(constant, 2), linear_h3)
            
            append_quadratic_safe((var_x, var_y), float(pow(constant, 2)) - 2*constant, quadratic_h3)
            
bqm_h3 = dimod.BinaryQuadraticModel(linear_h3, quadratic_h3, offset_h3, vartype)
bqm_h3.scale(A3)
main_bqm.update(bqm_h3)
    

### Constraint 3: minimizing total price

In [9]:
# Encoding constraint H4

def number_of_points_covered(sensor):
    # Following code calculates volume of view of sensor
    side1 = 2*sensor["view"]["range"]*float(math.tan(math.pi * float(sensor["view"]["angle"]["horizontal"] / 360)))
    side2 = 2*sensor["view"]["range"]*float(math.tan(math.pi * float(sensor["view"]["angle"]["vertical"] / 360)))
    return (1/3)*sensor["view"]["range"]*side1*side2

total_points = len(env)

A4 = 1
linear_h4 = {}
quadratic_h4 = {}
offset_h4 = float(pow(total_points, 2))

for sensor in sensors:
    for x in car_surface:
        for y in env:
            covered_area = number_of_points_covered(sensor)
            
            quadratic_constant = float(pow(covered_area, 2)) - 2*covered_area*total_points
            
            var_x = (x[0], x[1], x[2], sensor["id"])
            var_y = (y[0], y[1], y[2], sensor["id"])
            
            append_quadratic_safe((var_x, var_y), quadratic_constant, quadratic_h4)
            
bqm_h4 = dimod.BinaryQuadraticModel(linear_h4, quadratic_h4, offset_h4, vartype)
bqm_h4.scale(A4)
main_bqm.update(bqm_h4)
            

### Constraint 4: optimizing overlap of sensor views

In [10]:
# Encoding constraint H5

for p in env:
    cr = critical_value(p[0], p[1])
    A5 = 1
    linear_h5 = {}
    quadratic_h5 = {}
    offset_h5 = float(pow(cr, 2))
    
    for sensor in sensors:
        for x in car_surface:
            for y in env:
                covers = sensor_covers(x, y, p, sensor)
                
                var_x = (x[0], x[1], x[2], sensor["id"])
                var_y = (y[0], y[1], y[2], sensor["id"])
            
                append_quadratic_safe((var_x, var_y), float(pow(covers, 2) - 2*cr*covers), quadratic_h5)
                
    bqm_h5 = dimod.BinaryQuadraticModel(linear_h5, quadratic_h5, offset_h5, vartype)
    bqm_h5.scale(A5)
    main_bqm.update(bqm_h5)

### Constraint 5: minimizing total price

In [11]:
# Encoding constraint H6

A6 = 1
linear_h6 = {}
quadratic_h6 = {}
offset_h6 = 0.0

for sensor in sensors:
    for x in car_surface:
        for y in env:
            var_x = (x[0], x[1], x[2], sensor["id"])
            var_y = (y[0], y[1], y[2], sensor["id"])
            
            append_quadratic_safe((var_x, var_y), sensor["price"], quadratic_h6)

bqm_h6 = dimod.BinaryQuadraticModel(linear_h6, quadratic_h6, offset_h6, vartype)
bqm_h6.scale(A6)
main_bqm.update(bqm_h6)

#for elem in main_bqm.linear:
#        print(elem, main_bqm.linear[elem])

#for elem in main_bqm.quadratic:
#    if main_bqm.quadratic[elem] != 16:
#        print(elem, main_bqm.quadratic[elem])

### Solve QUBO

Unfortunately, LeapHybridSampler is not available in Amazon Bracket. That is why this code will not work in Bracket. On the other hand, we tried run the code using BracketDwaveSampler but the problem cannot be mapped to the circuits.

In [12]:
main_bqm.normalize()

sampler = LeapHybridSampler()
sampleset = sampler.sample(main_bqm)
sample = sampleset.first.sample
print(sampleset)
print()
# energy = sampleset.first.energy
print("Possible sensor positions in the space (x-coordinate, y-coordinate, sensor id):")
i = 0
for varname, value in sample.items():
    if value == 1:
        i+=1
        print(varname, value)
print(i)

  (-8, -8, 1) (-8, -8, 2) (-8, -8, 3) ... (8, 8, 5)    energy num_oc.
0           0           0           0 ...         0 -0.009236       1
['BINARY', 1 rows, 1 samples, 1425 variables]

Possible sensor positions in the space (x-coordinate, y-coordinate, sensor id):
(-8, -8, 5) 1
(-8, -7, 5) 1
(-8, -6, 5) 1
(-8, -5, 5) 1
(-8, -4, 5) 1
(-8, -3, 5) 1
(-8, -2, 5) 1
(-8, 4, 5) 1
(-8, 7, 5) 1
(-8, 8, 5) 1
(-7, -8, 5) 1
(-7, -7, 5) 1
(-7, -6, 5) 1
(-7, -5, 5) 1
(-7, -4, 5) 1
(-7, -3, 5) 1
(0, 0, 5) 1
(0, 1, 5) 1
(0, 2, 5) 1
(0, 3, 5) 1
(7, -8, 5) 1
(7, -7, 5) 1
(7, -6, 5) 1
(7, -5, 5) 1
(7, -4, 5) 1
(8, -8, 5) 1
(8, -7, 5) 1
(8, -6, 5) 1
(8, -5, 5) 1
(8, -4, 5) 1
30
